# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [49]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        if 1 - answer * self.f(x) > 0:
            der_0 = - answer # w0 derivative
            der_w = - answer * x # the rest derivatives vector
            return der_w, der_0
        else:
            return np.zeros_like(x), 0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k
            
            der_w, der_0 = self.der_loss(x, y)
            der_w += self.der_reg()

            # w update
            self.w -= step * der_w
            # w_0 update
            self.w0 -= step * der_0

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [40]:
from sklearn.metrics import classification_report

In [69]:
model = MySVM(C = 2000)
model.fit(X_train, y_train)

In [70]:
print(classification_report(y_train, model.predict(X_train)))

             precision    recall  f1-score   support

          0       0.66      0.51      0.58      3997
          1       0.60      0.74      0.67      4003

avg / total       0.63      0.63      0.62      8000



In [75]:
from sklearn.svm import LinearSVC
svc = LinearSVC(C=1)
svc.fit(X_train, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [76]:
print(classification_report(y_train, svc.predict(X_train)))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80      3997
          1       0.80      0.80      0.80      4003

avg / total       0.80      0.80      0.80      8000



MySVM нервно курит в сторонке; наверное он бы работал лучше, если веса обновлялись не по одному объекты а по случайным батчам.